In [1]:
# import sys
# !{sys.executable} -m pip install --upgrade tensorflow 
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import glacierml as gl
import numpy as np
import warnings
from tensorflow.python.util import deprecation
import os
import logging
import seaborn as sns
import janitor
import tabulate
from tqdm import tqdm
from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))
tf.get_logger().setLevel(logging.ERROR)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
deprecation._PRINT_DEPRECATION_WARNINGS = False
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

2022-06-01 20:35:04.230467: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
print(tf.__version__)


2.9.1


# Step 1.
### Load and define data

In [19]:
Glam = pd.read_csv('Glam.csv')
Glam = Glam[[
#         'LAT',
#         'LON',
    'CenLon',
    'CenLat',
    'Area',
    'thickness',
    'Slope',
    'Zmin',
    'Zmed',
    'Zmax',
    'Aspect',
    'Lmax'
]]
# split data for training and validation
(train_features, test_features, train_labels, test_labels) = gl.data_splitter(Glam)

# define model hyperparameters
RS = range(0,25,1)
ep = 300

# name databases
Glam.name = 'Glam'

# Step 2
### Load and models, then make predictions

In [20]:
rootdir = 'sm2/'
# print(rootdir)
dnn_model={}
predictions = pd.DataFrame()

for arch in tqdm(os.listdir(rootdir)):
    for folder in os.listdir(rootdir+arch):
        if 'MULTI' in folder and 'dnn' in folder:
            
            if '0.1' in folder:
                dnn_model[arch[3:]+'_'+folder] = tf.keras.models.load_model(rootdir 
                    + arch 
                    + '/' 
                    + folder)

                mae_test = dnn_model[arch[3:]+'_'+folder].evaluate(test_features,
                                                             test_labels,verbose=0)

                mae_train = dnn_model[arch[3:]+'_'+folder].evaluate(train_features,
                                             train_labels,verbose=0)

                pred_train = dnn_model[arch[3:]+'_'+folder].predict(train_features,verbose=0)

                pred_test = dnn_model[arch[3:]+'_'+folder].predict(test_features,verbose=0)
                avg_thickness = pd.Series((np.sum(pred_train) / len(pred_train)), name = 'avg train thickness')

                avg_test_thickness = pd.Series((np.sum(pred_test) / len(pred_test)),  name = 'avg test thickness')
                temp_df = pd.merge(avg_thickness, avg_test_thickness, right_index=True, left_index=True)
                predictions = predictions.append(temp_df, ignore_index=True)
                predictions.loc[predictions.index[-1], 'model']= folder
                predictions.loc[predictions.index[-1], 'test mae']= mae_test
                predictions.loc[predictions.index[-1], 'train mae']= mae_train
                predictions.loc[predictions.index[-1], 'architecture']= arch[3:]
                predictions.loc[predictions.index[-1], 'learning rate']= '0.1'
                predictions.loc[predictions.index[-1], 'validation split']= '0.2'
                
            if '0.01' in folder:
                dnn_model[arch[3:]+'_'+folder] = tf.keras.models.load_model(rootdir 
                    + arch 
                    + '/' 
                    + folder)

                mae_test = dnn_model[arch[3:]+'_'+folder].evaluate(test_features,
                                                             test_labels,verbose=0)

                mae_train = dnn_model[arch[3:]+'_'+folder].evaluate(train_features,
                                             train_labels,verbose=0)

                pred_train = dnn_model[arch[3:]+'_'+folder].predict(train_features, verbose=0)

                pred_test = dnn_model[arch[3:]+'_'+folder].predict(test_features,verbose=0)
                avg_thickness = pd.Series((np.sum(pred_train) / len(pred_train)), name = 'avg train thickness')

                avg_test_thickness = pd.Series((np.sum(pred_test) / len(pred_test)),  name = 'avg test thickness')
                temp_df = pd.merge(avg_thickness, avg_test_thickness, right_index=True, left_index=True)
                predictions = predictions.append(temp_df, ignore_index=True)
                predictions.loc[predictions.index[-1], 'model']= folder
                predictions.loc[predictions.index[-1], 'test mae']= mae_test
                predictions.loc[predictions.index[-1], 'train mae']= mae_train
                predictions.loc[predictions.index[-1], 'architecture']= arch[3:]
                predictions.loc[predictions.index[-1], 'learning rate']= '0.01'
                predictions.loc[predictions.index[-1], 'validation split']= '0.2'          
            
            if '0.001' in folder:
                dnn_model[arch[3:]+'_'+folder] = tf.keras.models.load_model(rootdir 
                    + arch 
                    + '/' 
                    + folder)

                mae_test = dnn_model[arch[3:]+'_'+folder].evaluate(test_features,
                                                             test_labels,verbose=0)

                mae_train = dnn_model[arch[3:]+'_'+folder].evaluate(train_features,
                                             train_labels,verbose=0)

                pred_train = dnn_model[arch[3:]+'_'+folder].predict(train_features,verbose=0)

                pred_test = dnn_model[arch[3:]+'_'+folder].predict(test_features,verbose=0)
                avg_thickness = pd.Series((np.sum(pred_train) / len(pred_train)), name = 'avg train thickness')

                avg_test_thickness = pd.Series((np.sum(pred_test) / len(pred_test)),  name = 'avg test thickness')
                temp_df = pd.merge(avg_thickness, avg_test_thickness, right_index=True, left_index=True)
                predictions = predictions.append(temp_df, ignore_index=True)
                predictions.loc[predictions.index[-1], 'model']= folder
                predictions.loc[predictions.index[-1], 'test mae']= mae_test
                predictions.loc[predictions.index[-1], 'train mae']= mae_train
                predictions.loc[predictions.index[-1], 'architecture']= arch[3:]            
                predictions.loc[predictions.index[-1], 'learning rate']= '0.001'
                predictions.loc[predictions.index[-1], 'validation split']= '0.2'                
                
predictions.rename(columns = {0:'avg train thickness'},inplace = True)

100%|██████████| 5/5 [03:15<00:00, 39.02s/it]


In [21]:
"""
Here we evaluate models and make predictions, then display the zults
"""


# these models are ridiculous, so we drop them.
# idx = predictions.index[predictions['architecture']=='64']
# predictions = predictions.drop(predictions.loc[idx].index)


"""
Here we compute for each layer architecture avg mae, mae std dev, and
prediction std dev.
"""
deviations = pd.DataFrame()
for architecture in list(predictions['architecture'].unique()):
    for learning_rate in list(predictions['learning rate'].unique()):
        # define temp dataframe for calculations that contains only one layer architecture
#         df = (predictions[predictions['architecture'] == architecture]) and (predictions[predictions['learning rate'] == str(learning_rate)])
        df = predictions[(predictions['architecture'] == architecture) & (predictions['learning rate' ]== learning_rate)]
#         break
#         print(df)
        # step 1: calculate mean of numbers
        test_mae_mean = np.sum(df['test mae']) / len(df) 

        diff_sq = pd.Series()

        for test_mae in df['test mae']:
            # step 2: subtract the mean from each, then square the result
            step_2 = pd.Series((test_mae - test_mae_mean)**2)
            diff_sq = diff_sq.append(step_2, ignore_index=True)

        # step 3: work out the mean of the squared differences    
        mean_diff_sq = (np.sum(diff_sq) / len(diff_sq))

        # step 4: take the square root
        test_mae_std_dev = np.sqrt(mean_diff_sq)


       # repeat for train mae 

        # step 1: calculate mean of numbers
        train_mae_mean = np.sum(df['train mae']) / len(df) 

        diff_sq = pd.Series()

        for train_mae in df['train mae']:
            # step 2: subtract the mean from each, then square the result
            step_2 = pd.Series((train_mae - train_mae_mean)**2)
            diff_sq = diff_sq.append(step_2, ignore_index=True)

        # step 3: work out the mean of the squared differences    
        mean_diff_sq = (np.sum(diff_sq) / len(diff_sq))

        # step 4: take the square root
        train_mae_std_dev = np.sqrt(mean_diff_sq)

        # repeat process for train thicknesses
        thickness_train_mean = np.sum(df['avg train thickness']) / len(df)   
        for thickness in df['avg train thickness']:
            step_2 = pd.Series((thickness - thickness_train_mean)**2)
            diff_sq = diff_sq.append(step_2, ignore_index=True)
        mean_diff_sq = (np.sum(diff_sq) / len(diff_sq))
        train_thickness_std_dev = np.sqrt(mean_diff_sq)


        # repeat process for test thicknesses
        thickness_test_mean = np.sum(df['avg test thickness']) / len(df)   
        for thickness in df['avg test thickness']:
            step_2 = pd.Series((thickness - thickness_test_mean)**2)
            diff_sq = diff_sq.append(step_2, ignore_index=True)
        mean_diff_sq = (np.sum(diff_sq) / len(diff_sq))
        test_thickness_std_dev = np.sqrt(mean_diff_sq)

        # turn the last number computed into a series so it may be appended to build the table.
        # it will be dropped later, no worries.
        test_thick_std_dev = pd.Series(test_thickness_std_dev)

        deviations = deviations.append(test_thick_std_dev, ignore_index=True)
        deviations.loc[deviations.index[-1], 'layer architecture']= architecture
        deviations.loc[deviations.index[-1], 'model parameters'] = dnn_model[architecture + '_Glam_dnn_MULTI_0.1_0.2_300_0'].count_params()
        deviations.loc[deviations.index[-1], 'learning rate']= learning_rate
        deviations.loc[deviations.index[-1], 'validation split']= 0.2
        deviations.loc[deviations.index[-1], 'test mae avg'] = test_mae_mean
        deviations.loc[deviations.index[-1], 'train mae avg']= train_mae_mean
        deviations.loc[deviations.index[-1], 'test mae std dev']= test_mae_std_dev
        deviations.loc[deviations.index[-1], 'train mae std dev']= train_mae_std_dev
        deviations.loc[deviations.index[-1], 'test predicted thickness std dev']= test_thickness_std_dev
        deviations.loc[deviations.index[-1], 'train predicted thickness std dev']= train_thickness_std_dev
# bootstrapped ensembles for predicted column    
#drop that appended line from earlier. Probably a better way to go about it
deviations.drop(columns = {0},inplace = True)    
deviations = deviations.dropna()
# deviations['training split'] = deviations['test mae avg'] - deviations['train mae avg']
# too_low = deviations.index[deviations['training split'] < 0]
# deviations = deviations.drop(too_low)
deviations = deviations.sort_values('test mae avg')
deviations

,layer architecture,model parameters,learning rate,validation split,test mae avg,train mae avg,test mae std dev,train mae std dev,test predicted thickness std dev,train predicted thickness std dev
8,32-16-8,1012.0,0.001,0.2,12.207832,13.195737,0.325927,0.427030,0.591357,0.486747
2,16-8,324.0,0.001,0.2,12.882613,14.671638,0.255216,0.218825,0.364422,0.352042
5,12-6,224.0,0.001,0.2,12.949542,15.135346,0.284604,0.285442,0.520103,0.430229
11,10-5,180.0,0.001,0.2,13.091524,15.287114,0.382745,0.327079,0.488009,0.434572
4,12-6,224.0,0.01,0.2,13.258951,12.697351,0.994132,0.708047,1.165205,0.954331
10,10-5,180.0,0.01,0.2,13.308620,12.831752,0.602335,0.627691,0.797038,0.746737
1,16-8,324.0,0.01,0.2,13.468252,11.856179,0.910006,0.837275,0.966888,0.882962
13,8-4,140.0,0.01,0.2,13.658732,13.962586,3.111561,3.332467,4.761145,4.271081
9,10-5,180.0,0.1,0.2,14.284519,11.670497,1.606948,1.948284,1.970232,1.917905
6,32-16-8,1012.0,0.1,0.2,14.459327,10.380450,1.225341,1.412582,4.272036,3.714475


# Build RGI 

In [22]:
rootdir = '/data/fast0/datasets/rgi60-attribs/'
RGI_extra = pd.DataFrame()
for file in os.listdir(rootdir):
    print(file)
    f = pd.read_csv(rootdir+file, encoding_errors = 'replace', on_bad_lines = 'skip')
    RGI_extra = RGI_extra.append(f, ignore_index = True)

RGI = RGI_extra[[
    'CenLat',
    'CenLon',
    'Slope',
    'Zmin',
    'Zmed',
    'Zmax',
    'Area',
    'Aspect',
    'Lmax'
]]
bad_zmed = RGI.loc[RGI['Zmed']<0].index
RGI = RGI.drop(bad_zmed)
bad_lmax = RGI.loc[RGI['Lmax']<0].index
RGI = RGI.drop(bad_lmax)
bad_slope = RGI.loc[RGI['Slope']<0].index
RGI = RGI.drop(bad_slope)
bad_aspect = RGI.loc[RGI['Aspect']<0].index
RGI = RGI.drop(bad_aspect)
RGI = RGI.reset_index()
RGI = RGI.drop('index', axis=1)


arch = deviations['layer architecture'].iloc[0]
lr = deviations['learning rate'].iloc[0]
vs = deviations['validation split'].iloc[0]



"""
Here we predict thicknesses across all 25 of the top performing ensemble model
"""
# dfs = pd.DataFrame()
# for rs in tqdm(RS):
#     df = deviations.iloc[:1]
#     s = pd.Series(dnn_model[str(arch) + '_Glam_dnn_MULTI_' + str(lr) + '_' + str(vs) + '_' +str(ep) + '_' + str(rs)].predict(RGI, verbose=0).flatten(), name = rs)
#     dfs[rs] = s
    
dfs = pd.DataFrame()
for rs in tqdm(RS):
    df = deviations.iloc[:1]
    s = pd.Series(dnn_model[str(arch) +'_Glam_dnn_MULTI_'+str(lr)+'_'+str(vs)+'_300_'+ str(rs)].predict(RGI, verbose=0).flatten(), name = rs)
    dfs[rs] = s
    
    
"""
Here we compute the average thickness for each RGI glacier
"""
RGI['avg predicted thickness'] = 'NaN'
for i in tqdm(dfs.index):
    avg_predicted_thickness = np.sum(dfs.loc[i])/len(dfs.loc[i])
#     print(i)
#     print(np.sum(dfs.loc[i])/len(dfs.loc[i]))
#     print('')
#     break
    RGI['avg predicted thickness'].loc[i] = avg_predicted_thickness

01_rgi60_Alaska.csv
02_rgi60_WesternCanadaUS.csv
03_rgi60_ArcticCanadaNorth.csv
04_rgi60_ArcticCanadaSouth.csv
05_rgi60_GreenlandPeriphery.csv
06_rgi60_Iceland.csv
07_rgi60_Svalbard.csv
08_rgi60_Scandinavia.csv
09_rgi60_RussianArctic.csv
10_rgi60_NorthAsia.csv
11_rgi60_CentralEurope.csv
12_rgi60_CaucasusMiddleEast.csv
13_rgi60_CentralAsia.csv
14_rgi60_SouthAsiaWest.csv
15_rgi60_SouthAsiaEast.csv
16_rgi60_LowLatitudes.csv
17_rgi60_SouthernAndes.csv
18_rgi60_NewZealand.csv
19_rgi60_AntarcticSubantarctic.csv


  0%|          | 0/208315 [00:00<?, ?it/s]/home/sa42/miniconda3/envs/python-cartopy-f/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|██████████| 208315/208315 [03:21<00:00, 1031.87it/s]


In [23]:
RGI['predicted thickness std dev'] = 'NaN'
for i in tqdm(dfs.index):
    # step 1: calculate mean of numbers
    avg_predicted_thickness = np.sum(dfs.loc[i])/len(dfs.loc[i])
    
    # step 2: subtract the mean from each, then square the result
    
    diff_sq = pd.Series()

    for q in dfs:
        
        avg_predicted_thickness - dfs[q].loc[i]
        step_2 = pd.Series((avg_predicted_thickness - dfs[q].loc[i])**2)
        diff_sq = diff_sq.append(step_2, ignore_index=True)
    
    # step 3: work out the mean of the squared differences    
    mean_diff_sq = (np.sum(diff_sq) / len(diff_sq))
    # step 4: take the square root
    prethick_std_dev = np.sqrt(mean_diff_sq)
    RGI['predicted thickness std dev'].loc[i] = prethick_std_dev


100%|██████████| 208315/208315 [19:16<00:00, 180.19it/s]


In [24]:
RGI['variance'] = (RGI['predicted thickness std dev'])**2

In [25]:
RGI

,CenLat,CenLon,Slope,Zmin,Zmed,Zmax,Area,Aspect,Lmax,avg predicted thickness,predicted thickness std dev,variance
0,63.6890,-146.823,42.0,1936,2385,2725,0.360,346,839,1289.063906,278.267929,77433.040075
1,63.4040,-146.668,16.0,1713,2005,2144,0.558,162,1197,1158.236172,241.924754,58527.58655
2,63.3760,-146.080,18.0,1609,1868,2182,1.685,175,2106,1083.203984,227.856378,51918.529149
3,63.3810,-146.120,19.0,1273,1944,2317,3.681,195,4175,845.136172,181.51366,32947.20882
4,63.5510,-147.057,16.0,1494,1914,2317,2.573,181,2981,1004.057344,211.500102,44732.293064
...,...,...,...,...,...,...,...,...,...,...,...,...
208310,-43.4215,170.354,34.7,1231,1724,2098,0.189,116,944,858.659922,181.499106,32941.925534
208311,-43.4550,170.349,36.1,1881,2106,2208,0.040,108,331,1315.604375,272.310819,74153.182011
208312,-43.4400,170.351,39.2,1677,1974,2253,0.184,104,740,1168.286641,244.716841,59886.332135
208313,-43.4106,170.364,34.0,1627,1839,1928,0.111,135,406,1139.257656,236.082618,55735.002431


In [26]:
print('predicted RGI sum thickness')
print(sum(RGI['avg predicted thickness']))
print('')
print('predicted RGI thickness variance')
print(sum(RGI['variance']))


predicted RGI sum thickness
442906059.7874193

predicted RGI thickness variance
53972755496.75347


In [ ]:
"""
This code plots predictions against actual thickness. Not currently modified and will load
EVERY SINGLE MODEL. DO NOT USE JUST YET
"""
for rs in RS:    
    y = dnn_model['10-5_Glam_dnn_MULTI_0.01_0.2_300_'+ str(rs)].predict(test_features, verbose=0)
    fig,ax=plt.subplots(1,1,figsize=(15,10))
    fig.patch.set_facecolor('w')
    plt.plot(test_labels,y,'.')
    plt.plot((0,300),(0,300),'-')
    plt.xlabel('True Thickness (m)')
    plt.ylabel('Model Thickness (m)')
    ax.set_title('Random State ' +str(rs))
    plt.xlim((0,300))
    plt.ylim((0,300))
    # plt.savefig("/home/sa42/notebooks/glac/figs/GTP_T")

In [ ]:
ccccombo_breaker()
# past this point is under construction. 
# here be monsters

In [ ]:
deviations

In [ ]:
# load all histories
print('Loading histories....')
rootdir = 'sr2/'
dnn_history = {}
arch = deviations['layer architecture'].iloc[[0]]
lr = deviations['learning rate'].iloc[[0]]
vs = deviations['validation split'].iloc[[0]]
print(arch)
print(lr)
print(vs)
# for arch in tqdm(os.listdir(rootdir)):
#     for folder in os.listdir(rootdir+arch):
#         if 'MULTI' in folder:
#             if 'dnn' in folder:

#                 dnn_history[arch[3:] + '_'+ folder] = pd.read_csv(rootdir+arch+'/'+folder)


"""
This cell plots each random state loss curve for the 25 random states for each run
Also loads EVERY SINGLE MODEL currently and blows up the memory. Working on it.
"""
# for rs in RS:
# for hist in dnn_history:    
#     print(hist)
#     fig,ax=plt.subplots(1,1,figsize=(10,10))
#     fig.patch.set_facecolor('w')
#     ax.set_title(hist)
#     gl.plot_loss(dnn_history[hist])
#     plt.savefig("/home/sa42/notebooks/glac/figs/GTP_dnn_loss.eps")

In [ ]:
"""
This cell plots the different variable loss curves to show individual variable models
Not currently working because we have only loaded dnn_MULTI models
"""
fig,ax=plt.subplots(2,2,figsize=(10,10))
fig.patch.set_facecolor('w')
# ax.set_ylim([5,30])

# gl.plot_loss(dnn_history['T_MULTI'])
for i, variable_name in enumerate(list(train_features)):
    ax = plt.subplot(2,2,i+1)
    gl.plot_loss(dnn_history['glacier_'+ variable_name+ '_0.1_0.2_300_6'])
#     ax.set_ylim([35,140])
    ax.set_title(variable_name)
    plt.tight_layout()
#     plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_dnn_loss.eps")


In [ ]:
predictions = predictions.rename(columns = {'architecture':'layer architecture'})

In [ ]:
dfs = pd.DataFrame()
for rs in RS:
    print(rs)
    df = deviations.iloc[:1]
    s = pd.Series(dnn_model['10-5_Glam_dnn_MULTI_0.01_0.2_300_'+str(rs)].predict(RGI).flatten(), name = rs)
    dfs[rs] = s
#     break
#     dfs = dfs.assign(str(s))
print(s)
dfs


In [ ]:
for model in predictions[(predictions['learning rate'] == df['learning rate']) and (predictions['layer architecture'] == df['layer architecture'])]:
    fig,ax = plt.subplots()
    ax.set_title('Layer architecture: ' + architecture)
    ax.set_ylabel('prediction count')
    ax.set_xlabel('thickness (m)')
    fig.patch.set_facecolor('w')
    plt.hist(df['avg test thickness'])
# print(df)

In [ ]:
for model in predictions['model']:
    print(model)

In [ ]:
# predictions['layer architecture']
# df['layer architecture']
# predictions['learning rate']
# df['learning rate']

In [ ]:
if predictions['layer architecture'].values() == [str(0.1)]:
    print(ding)

In [ ]:
# predictions['layer architecture']
# predictions[(predictions['learning rate'] == df['learning rate']) and (predictions['layer architecture'] == df['layer architecture'])]:
for architecture in predictions['layer architecture']:
#     if predictions['layer architecture'] == df['layer architecture'] and predictions['learning rate'] == df['learning rate']:
    print('')
    print(architecture)

In [ ]:
deviations = deviations.sort_values('test mae avg')
deviations

In [ ]:
"""
This cell contains code to produce histograms of all the architectures different histories
"""


df = deviations.iloc[:1]


for rs in RS:
    df = predictions[predictions['architecture'] == architecture]
    fig,ax = plt.subplots()
    ax.set_title('Layer architecture: ' + architecture)
    ax.set_ylabel('prediction count')
    ax.set_xlabel('thickness (m)')
    fig.patch.set_facecolor('w')
    plt.hist(df['avg test thickness'])
# print(df)

In [ ]:
# """
# This cell contains code to produce histograms of all the architectures different histories
# """

# for architecture in list(predictions['architecture'].unique()):
#     df = predictions[predictions['architecture'] == architecture]
#     fig,ax = plt.subplots()
#     ax.set_title('Layer architecture: ' + architecture)
#     ax.set_ylabel('prediction count')
#     ax.set_xlabel('thickness (m)')
#     fig.patch.set_facecolor('w')
#     plt.hist(df['avg test thickness'])
# # print(df)

In [ ]:
"""
This cell plots each random state loss curve for the 25 random states for each run
Also loads EVERY SINGLE MODEL currently and blows up the memory. Working on it.
"""
# for rs in RS:
for hist in dnn_history:    
    fig,ax=plt.subplots(1,1,figsize=(10,10))
    fig.patch.set_facecolor('w')
    ax.set_title(hist)
    gl.plot_loss(dnn_history[hist])
#     plt.savefig("/home/sa42/notebooks/glac/figs/GTP_dnn_loss.eps")

In [ ]:
# for modl in dnn_modl:    
#     y = dnn_modl[modl].predict(test_features)
#     fig,ax=plt.subplots(1,1,figsize=(15,10))
#     fig.patch.set_facecolor('w')
#     plt.plot(test_labels,y,'.')
#     plt.plot((0,300),(0,300),'-')
#     plt.xlabel('True Thickness (m)')
#     plt.ylabel('Model Thickness (m)')
#     plt.xlim((0,300))
#     plt.ylim((0,300))
#     # plt.savefig("/home/sa42/notebooks/glac/figs/GTP_T")

In [ ]:
# load all histories
print('Loading histories....')
rootdir = 'sr2/'
dnn_history = {}
for arch in tqdm(os.listdir(rootdir)):
    for folder in os.listdir(rootdir+arch):
        if 'MULTI' in folder:
            if 'dnn' in folder:

                dnn_history[arch[3:] + '_'+ folder] = pd.read_csv(rootdir+arch+'/'+folder)
# dnn_history

In [ ]:
dnn_history

In [ ]:
dnn_model

In [ ]:
"""
This is a density plot of the most recent predictions made. Can easily be modified in a loop
to show multiple random states and whatnot
"""
sns.set(rc={"figure.figsize":(15,10)})
sns.kdeplot(x = test_labels, y = y.flatten(),fill = True)
plt.plot((0,300),(0,300),'-')

In [ ]:
deviations.sort_values('test mae avg')


In [ ]:
dnn_history

In [ ]:
qqq.iloc[0]

In [ ]:
# # load all histories
# qqq = deviations.sort_values('test mae avg')


for rs in RS:
#     dnn_histor+y[qqq['layer architecture']+
#                 'Glam_dnn_history_MULTI_'+
#                 qqq['learning rate']+
#                 '_0.2_300_'+
#                 rs]
    gl.plot_loss(dnn_history[qqq['layer architecture'].iloc[0]+
                '_Glam_dnn_history_MULTI_'+
                qqq['learning rate'].iloc[0]+
                '_0.2_300_'+
                str(rs)])
# print('Loading histories....')
# rootdir = 'sr2/'
# dnn_history = {}
# for arch in tqdm(os.listdir(rootdir)):
#     for folder in os.listdir(rootdir+arch):
#         if 'MULTI' in folder and 'dnn' in folder:

#             dnn_history[arch[3:] + '_'+ folder] = pd.read_csv(rootdir+arch+'/'+folder)
# dnn_history

In [ ]:

[qqq['layer architecture'].iloc[0]+
                    '_Glam_dnn_history_MULTI_'+
                    qqq['learning rate'].iloc[0]+
                    '_0.2_300_'+
                    str(rs)]

In [ ]:
# def plot_single_variable(x,y,model_type,model_name,feature_name):
#     y = model_type[model_name].predict(test_labels)
#     plt.scatter(train_features[feature_name], train_labels, label='Data')
#     plt.plot(x, y,'.', color='k', label='Predictions')
#     plt.xlabel(feature_name)
#     plt.ylabel('THICKNESS')
#     plt.legend()
#     plt.plot()

# x = test_labels
# for i, variable_name in enumerate(list(train_features)):
#     ax = plt.subplot(2,2,i+1)
#     model_name = (dataset.name 
#     + '_' 
#     + variable_name 
#     + '_' 
#     + str(lr) 
#     + '_' 
#     + str(vs) 
#     + '_' 
#     + str(ep))
#     plot_single_variable(x,y,dnn_model, model_name,variable_name)
# #     ax.set_ylim([35,140])
# #     ax.set_title(variable_name)
# #     plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_dnn_loss.eps")
# plt.tight_layout()

In [ ]:
# """
# This cell contains code to load models and histories.
# """

# # fix hyperparameters 
# lr = 0.1
# vs = 0.2
# # load models
# print('Loading models....')

# linear_model = {}
# dnn_model = {}
# # data is already split, however if other databases are used, then this line is needed
# # (train_features, test_features, train_labels, test_labels) = gl.data_splitter(glacier)
# for variable_name in tqdm(list(train_features) + ['MULTI']):
#     for rs in RS:
#         file_name = (
#         pth_mod 
#         + 'glacier' 
#         + '_linear_' 
#         + variable_name 
#         + '_' 
#         + str(lr)
#         + '_'
#         + str(vs)
#         + '_'
#         + str(ep)
#         + '_'
#         + str(rs)
#         )

#         linear_model[
#             'glacier' 
#             + '_' 
#             + variable_name 
#             + '_' 
#             + str(lr)
#             + '_'
#             + str(vs)
#             + '_'
#             + str(ep)
#             + '_'
#             + str(rs)
#         ] = tf.keras.models.load_model(file_name)
        
# for variable_name in tqdm(list(train_features) + ['MULTI']):
#     for rs in RS:
#         file_name = (
#         pth_mod 
#         + 'glacier' 
#         + '_dnn_' 
#         + variable_name 
#         + '_' 
#         + str(lr)
#         + '_'
#         + str(vs)
#         + '_'
#         + str(ep)
#         + '_'
#         + str(rs)
#         )

#         dnn_model[
#             'glacier'
#             + '_' 
#             + variable_name 
#             + '_' 
#             + str(lr)
#             + '_'
#             + str(vs)
#             + '_'
#             + str(ep)
#             + '_'
#             + str(rs)
#         ] = tf.keras.models.load_model(file_name)
# print('Models loaded')

# # load all histories
# print('Loading histories....')
# linear_history = {}
# dnn_history = {}
# for variable_name in tqdm(list(train_features) + ['MULTI']):
#     for rs in RS:
#         file_name = (
#             pth_res 
#             + 'glacier' 
#             +'_linear_history_'
#             + variable_name 
#             + '_' 
#             + str(lr)
#             + '_'
#             + str(vs)
#             + '_'
#             + str(ep)
#             + '_'
#             + str(rs)
#         )

#         linear_history[
#             'glacier' 
#             +'_'
#             + variable_name 
#             + '_' 
#             + str(lr)
#             + '_'
#             + str(vs)
#             + '_'
#             + str(ep)
#             + '_'
#             + str(rs)
#         ]= pd.read_csv(file_name)

# for variable_name in tqdm(list(train_features) + ['MULTI']):
#     for rs in RS:
#         file_name = (
#             pth_res 
#             + 'glacier_dnn_history_' 
#             + variable_name 
#             + '_' 
#             + str(lr)
#             + '_'
#             + str(vs)
#             + '_'
#             + str(ep)
#             + '_'
#             + str(rs)
#         )

#         dnn_history[
#             'glacier' 
#             +'_'
#             + variable_name 
#             + '_' 
#             + str(lr)
#             + '_'
#             + str(vs)
#             + '_'
#             + str(ep)
#             + '_'
#             + str(rs)
#         ] = pd.read_csv(file_name)
# print('Histories loaded')

In [ ]:

"""
This cell loads the loss histories from the original model with one dense layer of 64 nodes.
From these histories I extracted the lowest and final loss value and plotted them.
First is plotted the loss from using different learning rates with a fixed validation split. 
Then follows a plot of different validation splits using a fixed learning rate.
"""

# set up dictionaries
loss = {}
dnn_lr_history = {}
loss['glacier_min_learn'] = pd.DataFrame()
loss['glacier_fin_learn'] = pd.DataFrame()

# loop to define and then load histories
for lr in LR:
    file_name = (
    'sr/sr_64/'
    + 'glacier_dnn_history_MULTI_'
    + str(lr)
    + '_'
    + str(vs)
    + '_'
    + str(ep))
    
    file = (
    'glacier_MULTI_'
    + str(lr)
    + '_'
    + str(vs)
    + '_'
    + str(ep))
    
    
    
    dnn_lr_history[
    'glacier_MULTI_' 
    + str(lr)
    + '_'
    + str(vs)
    + '_'
    + str(ep)
    ] = pd.read_csv(file_name)
    
    # find minimum and insert other model hyperparameters into table
    m_loss = dnn_lr_history[file].min()
    m_loss['learning rate'] = str(lr)
    m_loss['validation split'] = str(vs)
    m_loss['epochs'] = str(ep)
    loss['glacier_min_learn'] = loss['glacier_min_learn'].append(m_loss,ignore_index=True)
    
    
    # find final and insert other model hyperparameters into table
    f = dnn_lr_history[file].last_valid_index()
    f_loss = dnn_lr_history[file].iloc[[f]]
    f_loss['learning rate'] = str(lr)
    f_loss['validation split'] = str(vs)
    f_loss['epochs'] = str(ep)

    loss['glacier_fin_learn'] = loss['glacier_fin_learn'].append(f_loss,ignore_index=True)

loss['glacier_fin_learn'] = loss['glacier_fin_learn'].rename(columns = {
    'loss':'loss_final',
    'val_loss':'val_loss_final'
})

loss['glacier_min_learn'] = loss['glacier_min_learn'].rename(columns = {
    'loss':'loss_minimum',
    'val_loss':'val_loss_minimum'
})
    
print('Results compiled')
sns.set(rc={"figure.figsize":(15,10)})
ax = plt.gca()
ax.set_ylim([5,30])
# loss['glacier_min'].plot(x='validation split', y = ['loss_minimum', 'val_loss_minimum'], kind='bar',  ax=ax)
# loss['glacier_fin'].plot(x='validation split', y = ['loss_final', 'val_loss_final'], kind='bar',  ax=ax)

loss['glacier_fin_learn'].plot(x='learning rate', y = 'loss_final',color = 'blue',  ax=ax)
loss['glacier_fin_learn'].plot(x='learning rate', y = 'val_loss_final',color = 'green', ax=ax)
loss['glacier_min_learn'].plot(x='learning rate', y = 'loss_minimum', color = 'red', ax=ax)
loss['glacier_min_learn'].plot(x='learning rate', y = 'val_loss_minimum',color = 'orange', ax=ax)
ax.set_xlabel('Learning rate at fixed validation split = 0.2')
ax.set_ylabel('Mean Absolute Error')
ax.set_title('GlaThiDa Glacier scale dataset multivariable regression hyperparameterization')
plt.show()
plt.tight_layout()



# repeat above loop with fixed lr and varied vs
lr = 0.1
loss = {}
dnn_vs_history = {}
loss['glacier_min_valsplit'] = pd.DataFrame()
loss['glacier_fin_valsplit'] = pd.DataFrame()
for vs in VS:
    file_name = (
    'sr/sr_64/'
    + 'glacier_dnn_history_MULTI_'
    + str(lr)
    + '_'
    + str(vs)
    + '_'
    + str(ep))
    
    file = (
    'glacier_MULTI_'
    + str(lr)
    + '_'
    + str(vs)
    + '_'
    + str(ep))
    
    
    
    dnn_lr_history[
    'glacier_MULTI_' 
    + str(lr)
    + '_'
    + str(vs)
    + '_'
    + str(ep)
    ] = pd.read_csv(file_name)
    
    
    m_loss = dnn_lr_history[file].min()
    m_loss['learning rate'] = str(lr)
    m_loss['validation split'] = str(vs)
    m_loss['epochs'] = str(ep)
    loss['glacier_min_valsplit'] = loss['glacier_min_valsplit'].append(m_loss,ignore_index=True)

    f = dnn_lr_history[file].last_valid_index()
    f_loss = dnn_lr_history[file].iloc[[f]]
    f_loss['learning rate'] = str(lr)
    f_loss['validation split'] = str(vs)
    f_loss['epochs'] = str(ep)

    loss['glacier_fin_valsplit'] = loss['glacier_fin_valsplit'].append(f_loss,ignore_index=True)

loss['glacier_fin_valsplit'] = loss['glacier_fin_valsplit'].rename(columns = {
    'loss':'loss_final',
    'val_loss':'val_loss_final'
})

loss['glacier_min_valsplit'] = loss['glacier_min_valsplit'].rename(columns = {
    'loss':'loss_minimum',
    'val_loss':'val_loss_minimum'
})
    
print('Results compiled')
sns.set(rc={"figure.figsize":(15,10)})
ax = plt.gca()
ax.set_ylim([5,30])
# loss['glacier_min'].plot(x='validation split', y = ['loss_minimum', 'val_loss_minimum'], kind='bar',  ax=ax)
# loss['glacier_fin'].plot(x='validation split', y = ['loss_final', 'val_loss_final'], kind='bar',  ax=ax)

loss['glacier_fin_valsplit'].plot(x='validation split', y = 'loss_final',color = 'blue',  ax=ax)
loss['glacier_fin_valsplit'].plot(x='validation split', y = 'val_loss_final',color = 'green', ax=ax)
loss['glacier_min_valsplit'].plot(x='validation split', y = 'loss_minimum', color = 'red', ax=ax)
loss['glacier_min_valsplit'].plot(x='validation split', y = 'val_loss_minimum',color = 'orange', ax=ax)
ax.set_xlabel('Validation splits with learning rate = 0.1')
ax.set_ylabel('Mean Absolute Error')
ax.set_title('GlaThiDa Glacier scale dataset multivariable regression hyperparameterization')
plt.show()
plt.tight_layout()

In [ ]:
"""
This cell computes the true average thickness of the glaciers in use
"""


pth = '/data/fast1/glacierml/T_models/'
T_lab = pd.read_csv(pth + 'T.csv', low_memory = False)
T_lab = T_lab[[
    'GlaThiDa_ID',
    'LAT',
    'LON',
    'AREA',
    'MEAN_SLOPE',
    'MEAN_THICKNESS'
]]
T_lab = T_lab.dropna()

tru_thickness = np.sum(T_lab['MEAN_THICKNESS']) / len(T_lab['MEAN_THICKNESS'])
tru_thickness

In [ ]:
# set definitions
glacier = gl.data_loader(pth = '/data/fast1/glacierml/T_models/')
# ,TT,TTT,TTTx,TTT_full
# T_t = T.head()

# rename thickness column in database
gl.thickness_renamer(glacier)

# split data for training and validation
(train_features, test_features, train_labels, test_labels) = gl.data_splitter(glacier)

# define model hyperparameters
LR = np.logspace(-3,2,6)
vs = 0.2
VS = 0.1,0.15,0.2,0.25,0.3,0.35,0.4
RS = range(0,25,1)
ep = 300

# name databases
glacier.name = 'glacier'
# T_t.name = 'T_t'
# TT.name = 'band'
# TTT.name = 'point'
# TTTx.name = 'TTTx'
# TTT_full.name = 'TTT_full'

# old definitions, legacy code.

# arch = '16-8'
# pth_mod = 'sm/sm_' + arch + '/'
# pth_res = 'sr/sr_' + arch + '/'

"""
Here we evaluate models and make predictions, then display the zults
"""
rootdir = 'sm/'
# print(rootdir)
dnn_model = {}
predictions = pd.DataFrame()
for arch in tqdm(os.listdir(rootdir)):
    for folder in os.listdir(rootdir+arch):
        if 'MULTI' in folder and 'dnn' in folder:
            
            if '0.1' in folder:
                dnn_model[arch[3:]+'_'+folder] = tf.keras.models.load_model(rootdir 
                    + arch 
                    + '/' 
                    + folder)

                mae_test = dnn_model[arch[3:]+'_'+folder].evaluate(test_features,
                                                             test_labels,verbose=0)

                mae_train = dnn_model[arch[3:]+'_'+folder].evaluate(train_features,
                                             train_labels,verbose=0)

                pred_train = dnn_model[arch[3:]+'_'+folder].predict(train_features,verbose=0)

                pred_test = dnn_model[arch[3:]+'_'+folder].predict(test_features,verbose=0)
                avg_thickness = pd.Series((np.sum(pred_train) / len(pred_train)), name = 'avg train thickness')

                avg_test_thickness = pd.Series((np.sum(pred_test) / len(pred_test)),  name = 'avg test thickness')
                temp_df = pd.merge(avg_thickness, avg_test_thickness, right_index=True, left_index=True)
                predictions = predictions.append(temp_df, ignore_index=True)
                predictions.loc[predictions.index[-1], 'model']= folder
                predictions.loc[predictions.index[-1], 'test mae']= mae_test
                predictions.loc[predictions.index[-1], 'train mae']= mae_train
                predictions.loc[predictions.index[-1], 'architecture']= arch[3:]
                predictions.loc[predictions.index[-1], 'learning rate']= '0.1'
                predictions.loc[predictions.index[-1], 'validation split']= '0.2'
                
            if '0.01' in folder:
                dnn_model[arch[3:]+'_'+folder] = tf.keras.models.load_model(rootdir 
                    + arch 
                    + '/' 
                    + folder)

                mae_test = dnn_model[arch[3:]+'_'+folder].evaluate(test_features,
                                                             test_labels,verbose=0)

                mae_train = dnn_model[arch[3:]+'_'+folder].evaluate(train_features,
                                             train_labels,verbose=0)

                pred_train = dnn_model[arch[3:]+'_'+folder].predict(train_features, verbose=0)

                pred_test = dnn_model[arch[3:]+'_'+folder].predict(test_features,verbose=0)
                avg_thickness = pd.Series((np.sum(pred_train) / len(pred_train)), name = 'avg train thickness')

                avg_test_thickness = pd.Series((np.sum(pred_test) / len(pred_test)),  name = 'avg test thickness')
                temp_df = pd.merge(avg_thickness, avg_test_thickness, right_index=True, left_index=True)
                predictions = predictions.append(temp_df, ignore_index=True)
                predictions.loc[predictions.index[-1], 'model']= folder
                predictions.loc[predictions.index[-1], 'test mae']= mae_test
                predictions.loc[predictions.index[-1], 'train mae']= mae_train
                predictions.loc[predictions.index[-1], 'architecture']= arch[3:]
                predictions.loc[predictions.index[-1], 'learning rate']= '0.01'
                predictions.loc[predictions.index[-1], 'validation split']= '0.2'          
            
            if '0.001' in folder:
                dnn_model[arch[3:]+'_'+folder] = tf.keras.models.load_model(rootdir 
                    + arch 
                    + '/' 
                    + folder)

                mae_test = dnn_model[arch[3:]+'_'+folder].evaluate(test_features,
                                                             test_labels,verbose=0)

                mae_train = dnn_model[arch[3:]+'_'+folder].evaluate(train_features,
                                             train_labels,verbose=0)

                pred_train = dnn_model[arch[3:]+'_'+folder].predict(train_features,verbose=0)

                pred_test = dnn_model[arch[3:]+'_'+folder].predict(test_features,verbose=0)
                avg_thickness = pd.Series((np.sum(pred_train) / len(pred_train)), name = 'avg train thickness')

                avg_test_thickness = pd.Series((np.sum(pred_test) / len(pred_test)),  name = 'avg test thickness')
                temp_df = pd.merge(avg_thickness, avg_test_thickness, right_index=True, left_index=True)
                predictions = predictions.append(temp_df, ignore_index=True)
                predictions.loc[predictions.index[-1], 'model']= folder
                predictions.loc[predictions.index[-1], 'test mae']= mae_test
                predictions.loc[predictions.index[-1], 'train mae']= mae_train
                predictions.loc[predictions.index[-1], 'architecture']= arch[3:]            
                predictions.loc[predictions.index[-1], 'learning rate']= '0.001'
                predictions.loc[predictions.index[-1], 'validation split']= '0.2'                
                
predictions.rename(columns = {0:'avg train thickness'},inplace = True)

# these models are ridiculous, so we drop them.
# idx = predictions.index[predictions['architecture']=='64']
# predictions = predictions.drop(predictions.loc[idx].index)


"""
Here we compute for each layer architecture avg mae, mae std dev, and
prediction std dev.
"""
deviations = pd.DataFrame()
for architecture in list(predictions['architecture'].unique()):
    for learning_rate in list(predictions['learning rate'].unique()):
        # define temp dataframe for calculations that contains only one layer architecture
#         df = (predictions[predictions['architecture'] == architecture]) and (predictions[predictions['learning rate'] == str(learning_rate)])
        df = predictions[(predictions['architecture'] == architecture) & (predictions['learning rate' ]== learning_rate)]
#         break
#         print(df)
        # step 1: calculate mean of numbers
        test_mae_mean = np.sum(df['test mae']) / len(df) 

        diff_sq = pd.Series()

        for test_mae in df['test mae']:
            # step 2: subtract the mean from each, then square the result
            step_2 = pd.Series((test_mae - test_mae_mean)**2)
            diff_sq = diff_sq.append(step_2, ignore_index=True)

        # step 3: work out the mean of the squared differences    
        mean_diff_sq = (np.sum(diff_sq) / len(diff_sq))

        # step 4: take the square root
        test_mae_std_dev = np.sqrt(mean_diff_sq)


       # repeat for train mae 

        # step 1: calculate mean of numbers
        train_mae_mean = np.sum(df['train mae']) / len(df) 

        diff_sq = pd.Series()

        for train_mae in df['train mae']:
            # step 2: subtract the mean from each, then square the result
            step_2 = pd.Series((train_mae - train_mae_mean)**2)
            diff_sq = diff_sq.append(step_2, ignore_index=True)

        # step 3: work out the mean of the squared differences    
        mean_diff_sq = (np.sum(diff_sq) / len(diff_sq))

        # step 4: take the square root
        train_mae_std_dev = np.sqrt(mean_diff_sq)

        # repeat process for train thicknesses
        thickness_train_mean = np.sum(df['avg train thickness']) / len(df)   
        for thickness in df['avg train thickness']:
            step_2 = pd.Series((thickness - thickness_train_mean)**2)
            diff_sq = diff_sq.append(step_2, ignore_index=True)
        mean_diff_sq = (np.sum(diff_sq) / len(diff_sq))
        train_thickness_std_dev = np.sqrt(mean_diff_sq)


        # repeat process for test thicknesses
        thickness_test_mean = np.sum(df['avg test thickness']) / len(df)   
        for thickness in df['avg test thickness']:
            step_2 = pd.Series((thickness - thickness_test_mean)**2)
            diff_sq = diff_sq.append(step_2, ignore_index=True)
        mean_diff_sq = (np.sum(diff_sq) / len(diff_sq))
        test_thickness_std_dev = np.sqrt(mean_diff_sq)

        # turn the last number computed into a series so it may be appended to build the table.
        # it will be dropped later, no worries.
        test_thick_std_dev = pd.Series(test_thickness_std_dev)

        deviations = deviations.append(test_thick_std_dev, ignore_index=True)
        deviations.loc[deviations.index[-1], 'layer architecture']= architecture
        deviations.loc[deviations.index[-1], 'model parameters'] = dnn_model[architecture + '_glacier_dnn_MULTI_0.1_0.2_300_0'].count_params()
        deviations.loc[deviations.index[-1], 'learning rate']= learning_rate
        deviations.loc[deviations.index[-1], 'validation split']= 0.2
        deviations.loc[deviations.index[-1], 'test mae avg'] = test_mae_mean
        deviations.loc[deviations.index[-1], 'train mae avg']= train_mae_mean
        deviations.loc[deviations.index[-1], 'test mae std dev']= test_mae_std_dev
        deviations.loc[deviations.index[-1], 'train mae std dev']= train_mae_std_dev
        deviations.loc[deviations.index[-1], 'test predicted thickness std dev']= test_thickness_std_dev
        deviations.loc[deviations.index[-1], 'train predicted thickness std dev']= train_thickness_std_dev
# bootstrapped ensembles for predicted column    
#drop that appended line from earlier. Probably a better way to go about it
deviations.drop(columns = {0},inplace = True)    
deviations = deviations.dropna()
deviations.sort_values('test mae avg')

In [ ]:
predictions